In [1]:
import os
if os.name=='nt':
    try:
        mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
        os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
    except:
        pass
    
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm

tqdm.tqdm.pandas()

%matplotlib inline

In [2]:
df = pd.read_csv('../input/train_ver2.csv')
df2 = df.copy()

C:\Users\lyaa\AppData\Local\Continuum\miniconda3\envs\kaggle\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ult_null = pd.read_hdf('../input/data.hdf', 'train_ult_null_count')

In [4]:
ncodpers_mixnull = ult_null.loc[(ult_null.count_ult_notnull>0) & (ult_null.count_ult_null>0)].index.tolist()

In [5]:
df.ult_fec_cli_1t = pd.to_datetime(df.ult_fec_cli_1t)
df.fecha_dato = pd.to_datetime(df.fecha_dato)
df.fecha_alta = pd.to_datetime(df.fecha_alta)

In [6]:
df.sort_values(['ncodpers', 'fecha_dato'], inplace=True)

In [7]:
df.loc[df.ncodpers.isin(ncodpers_mixnull), 'ult_fec_cli_1t'] = \
    df.loc[df.ncodpers.isin(ncodpers_mixnull), 'ult_fec_cli_1t'].fillna(method='bfill')

In [8]:
df.loc[df.ncodpers.isin(ncodpers_mixnull), ['ncodpers', 'ult_fec_cli_1t']].head()

,ncodpers,ult_fec_cli_1t
4078994,15891,2015-08-05
5319232,15891,2015-08-05
4079035,16136,2016-03-17
5319290,16136,2016-03-17
5705240,16136,2016-03-17


In [9]:
df.age = pd.to_numeric(df.age, errors='coerce')

In [10]:
df.antiguedad = pd.to_numeric(df.antiguedad, errors='coerce')

In [11]:
fecha_dato = pd.DatetimeIndex(df.fecha_dato)
fecha_alta = pd.DatetimeIndex(df.fecha_alta)
fecha_diff = (fecha_dato.year-fecha_alta.year)*12 + (fecha_dato.month-fecha_alta.month)

In [12]:
df.indrel_1mes.value_counts(dropna=False).index.tolist()

[1.0, '1.0', '1', nan, '3.0', 3.0, '3', 'P', '2', '2.0', 2.0, '4.0', 4.0, '4']

In [13]:
indrel_1mes_map = {1.0:1, '1.0':1, '1':1, 'nan':np.nan, 
    '3.0':3, 3.0:3, '3':3, 'P':0, '2':2, '2.0':2, 2.0:2, '4.0':4, 4.0:4, '4':4}

In [14]:
df.indrel_1mes = df.indrel_1mes.map(indrel_1mes_map)

In [15]:
df.indrel_1mes.value_counts(dropna=False)

 1.0    13490681
NaN       149781
 3.0        4350
 2.0        1317
 0.0         874
 4.0         306
Name: indrel_1mes, dtype: int64

In [16]:
df.tiprel_1mes.value_counts(dropna=False)

I      7304875
A      6187123
NaN     149781
P         4656
R          870
N            4
Name: tiprel_1mes, dtype: int64

In [17]:
df.conyuemp.value_counts(dropna=False)

NaN    13645501
N          1791
S            17
Name: conyuemp, dtype: int64

In [18]:
id_null = df.loc[df.ind_empleado.isnull()].index.tolist()

In [19]:
ncodpers_null = df.loc[df.ind_empleado.isnull()].ncodpers.unique().tolist()

In [20]:
len(ncodpers_null)

7340

In [21]:
len(id_null)

27734

In [22]:
a = df.loc[df.ncodpers.isin(ncodpers_null)].groupby('ncodpers').ind_empleado.apply(lambda x: len(x) - x.isnull().sum())
a = pd.DataFrame(a)

In [23]:
b = df.loc[df.ncodpers.isin(ncodpers_null)].groupby('ncodpers').ind_empleado.apply(lambda x: len(x))
b = pd.DataFrame(b)

In [24]:
b.columns = ['nrow']

In [25]:
a.columns = ['nrow_info']

In [26]:
a = a.join(b)

In [27]:
a.groupby(['nrow_info', 'nrow'])['nrow'].count()

nrow_info  nrow
0          1       1264
           2        976
           3        765
           4        729
           5       1524
           6       1773
1          2          9
           3          5
           4          2
           5          5
           6          5
           7          8
2          3          7
           4          3
           5          4
           6          2
           7          4
           8          2
3          4          8
           5          4
           6          7
           7          1
           8          3
           9          5
4          5          2
           6          3
           7          7
           8          6
           9          4
           10         6
                   ... 
6          7          8
           8          4
           9          1
           10         2
           11         6
           12         3
7          8          7
           9          6
           10         5
           11         5


What feature does not change?

In [28]:
df.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')

In [29]:
ind_empleado = df.groupby('ncodpers').ind_empleado.progress_apply(lambda x: x.nunique(dropna=False))

100%|████████████████████████████████████████████████████████████████████████| 956645/956645 [01:39<00:00, 9627.76it/s]


In [30]:
col_unchange = ['pais_residencia', 'sexo', 'fecha_alta', 
                'indresi', 'indext', 'canal_entrada', 
                'tipodom', 'cod_prov', 'nomprov', 'renta'] # features that should not change over time

In [31]:
cols = ['ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento']

u = {}
for c in tqdm.tqdm_notebook(cols):
    u[c] = df.groupby('ncodpers')[c].progress_apply(lambda x: x.nunique(dropna=False))

100%|████████████████████████████████████████████████████████████████████████| 956645/956645 [01:40<00:00, 9522.84it/s]


Find features that are almost unique for each customer.

These features have 1 unique value for most of the customers and 2 values for very few customers. 

In [49]:
u_two = []
for c in tqdm.tqdm_notebook(cols):
    if len(u[c].value_counts())==2:
        u_two.append(c)

In [50]:
u_two

['ind_empleado',
 'pais_residencia',
 'sexo',
 'fecha_alta',
 'indresi',
 'indext',
 'conyuemp',
 'indfall',
 'tipodom',
 'renta']

`ind_empleado` customers with two values always have 'N' and NAN. So we can replace all NAN with 'N'

In [58]:
a = u['ind_empleado'].loc[u['ind_empleado']==2].index.tolist()
df.loc[df.ncodpers.isin(a), 'ind_empleado'].unique()

array([nan, 'N'], dtype=object)

In [59]:
df.ind_empleado = df.ind_empleado.fillna('N')

`pais_residencia` the residence country, check if everyone with two values has NAN

In [64]:
c = 'pais_residencia'
a = u[c].loc[u[c]==2].index.tolist()
a = df.loc[df.ncodpers.isin(a)].groupby('ncodpers')[c].apply(lambda x: x.isnull().any())

In [67]:
df.loc[df.ncodpers==36435]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
414724,2015-01-28,36435,N,ES,V,48.0,1996-04-10,0.0,231.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
838757,2015-02-28,36435,N,ES,V,48.0,1996-04-10,0.0,231.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
1460178,2015-03-28,36435,N,ES,V,48.0,1996-04-10,0.0,231.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,0
2094683,2015-04-28,36435,N,ES,V,48.0,1996-04-10,0.0,231.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
2931191,2015-05-28,36435,N,ES,V,48.0,1996-04-10,0.0,231.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
3563346,2015-06-28,36435,N,ES,V,48.0,1996-04-10,0.0,231.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
4087010,2015-07-28,36435,N,ES,V,48.0,1996-04-10,0.0,231.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
5321521,2015-08-28,36435,N,ES,V,48.0,1996-04-10,0.0,232.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
5707321,2015-09-28,36435,N,ES,V,48.0,1996-04-10,0.0,233.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
6902055,2015-10-28,36435,N,US,V,48.0,1996-04-10,0.0,234.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1


The example shows that the customer changes the residence country. So we can fix `pais_residencia` only if there is NAN and another valid value.

In [83]:
def fix_pais_residencia(x):
    x = x.fillna(method='bfill')
    x = x.fillna(method='ffill')
    return x

# df['pais_residencia'] = df.groupby('ncodpers')['pais_residencia'].progress_apply(fix_pais_residencia)

n = df.loc[df.ncodpers.isin(a[a].index)].groupby('ncodpers')['pais_residencia'].apply(fix_pais_residencia)

df.loc[df.ncodpers.isin(a[a].index), 'pais_residencia'] = n

In [93]:
a.head()

ncodpers
35152     True
36435    False
41449     True
47550    False
48623     True
Name: pais_residencia, dtype: bool

In [96]:
df.loc[df.ncodpers==41449]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
413900,2015-01-28,41449,N,ES,NaN,NaN,NaT,NaN,NaN,NaN,...,0,0,0,1,0,0,0,NaN,NaN,0
840054,2015-02-28,41449,N,ES,NaN,NaN,NaT,NaN,NaN,NaN,...,0,0,0,1,0,0,0,NaN,NaN,0
1461598,2015-03-28,41449,N,ES,NaN,NaN,NaT,NaN,NaN,NaN,...,0,0,0,1,0,0,0,NaN,NaN,0
2096108,2015-04-28,41449,N,ES,NaN,NaN,NaT,NaN,NaN,NaN,...,0,0,0,1,0,0,0,NaN,NaN,0
2929915,2015-05-28,41449,N,ES,NaN,NaN,NaT,NaN,NaN,NaN,...,0,0,0,1,0,0,0,NaN,NaN,0
3561914,2015-06-28,41449,N,ES,NaN,NaN,NaT,NaN,NaN,NaN,...,0,0,0,1,0,0,0,NaN,NaN,0
7484041,2015-11-28,41449,N,ES,V,58.0,1998-07-29,1.0,0.0,1.0,...,0,0,0,1,0,0,0,0.0,0.0,0
8630067,2015-12-28,41449,N,ES,V,58.0,1998-07-29,0.0,183.0,1.0,...,0,0,0,1,0,1,0,0.0,0.0,0
9408292,2016-01-28,41449,N,ES,V,58.0,1998-07-29,0.0,184.0,1.0,...,0,0,0,1,0,1,0,0.0,0.0,0
10417381,2016-02-28,41449,N,ES,V,58.0,1998-07-29,0.0,185.0,1.0,...,0,0,0,1,0,1,0,0.0,0.0,0


`sexo`, does it change between man and woman? Yes, there are 46 customers whose gender changes. I think this is a typo, because real gender change is rare, I guess.

In [101]:
c = 'sexo'
a = u[c].loc[u[c]==2].index.tolist()
a = df.loc[df.ncodpers.isin(a)].groupby('ncodpers')[c].apply(lambda x: x.isnull().any())

In [104]:
a.value_counts() # Some customer change gender?

True     309
False     46
Name: sexo, dtype: int64

In [105]:
a.tail()

ncodpers
1445017    False
1457392    False
1459058    False
1463211    False
1499273    False
Name: sexo, dtype: bool

In [109]:
df.loc[df.ncodpers==1459058].T

,6280955,6638857,7525393,8879888,9078343,10097316,10877054,12556554,13414360
fecha_dato,2015-09-28 00:00:00,2015-10-28 00:00:00,2015-11-28 00:00:00,2015-12-28 00:00:00,2016-01-28 00:00:00,2016-02-28 00:00:00,2016-03-28 00:00:00,2016-04-28 00:00:00,2016-05-28 00:00:00
ncodpers,1459058,1459058,1459058,1459058,1459058,1459058,1459058,1459058,1459058
ind_empleado,N,N,N,N,N,N,N,N,N
pais_residencia,ES,ES,ES,ES,ES,ES,ES,ES,ES
sexo,H,V,V,V,V,V,V,V,V
age,22,22,22,22,22,23,23,23,23
fecha_alta,2015-09-23 00:00:00,2015-09-23 00:00:00,2015-09-23 00:00:00,2015-09-23 00:00:00,2015-09-23 00:00:00,2015-09-23 00:00:00,2015-09-23 00:00:00,2015-09-23 00:00:00,2015-09-23 00:00:00
ind_nuevo,1,1,1,1,1,1,0,0,0
antiguedad,0,1,2,3,4,5,6,7,8
indrel,1,1,1,1,1,1,1,1,1


For customers with both NAN and valid value in `sexo`

In [111]:
def fix_sexo(x):
    x = x.fillna(method='bfill')
    x = x.fillna(method='ffill')
    return x

# df['pais_residencia'] = df.groupby('ncodpers')['pais_residencia'].progress_apply(fix_pais_residencia)

n = df.loc[df.ncodpers.isin(a[a].index)].groupby('ncodpers')['sexo'].apply(fix_sexo)

df.loc[df.ncodpers.isin(a[a].index), 'sexo'] = n

For customers with both V and H, use the latest record to fix the previous ones

In [119]:
a.tail()

ncodpers
1445017    False
1457392    False
1459058    False
1463211    False
1499273    False
Name: sexo, dtype: bool

In [124]:
def fix_sexo2(x):
    x.iloc[:] = x.iloc[-1]    
    return x

n = df.loc[df.ncodpers.isin(a[~a].index)].groupby('ncodpers')['sexo'].apply(fix_sexo2)

In [126]:
df.loc[df.ncodpers.isin(a[~a].index), 'sexo'] = n

Look at the 

In [128]:
df.isnull().sum()

fecha_dato                      0
ncodpers                        0
ind_empleado                    0
pais_residencia             26685
sexo                        26755
age                         27734
fecha_alta                  27734
ind_nuevo                   27734
antiguedad                  27734
indrel                      27734
ult_fec_cli_1t           13518739
indrel_1mes                149781
tiprel_1mes                149781
indresi                     27734
indext                      27734
conyuemp                 13645501
canal_entrada              186126
indfall                     27734
tipodom                     27735
cod_prov                    93591
nomprov                     93591
ind_actividad_cliente       27734
renta                     2794375
segmento                   189368
ind_ahor_fin_ult1               0
ind_aval_fin_ult1               0
ind_cco_fin_ult1                0
ind_cder_fin_ult1               0
ind_cno_fin_ult1                0
ind_ctju_fin_u